In [11]:
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

import pickle
import pandas as pd
import time

from utils import greedy, simplex_projection, integer_initial_vector
from continuous import solve_continuous_quadratic

In [12]:
from utils import projection

n = 100
setting = f'n{n}'

for obj in ['f', 'crash', 'fuel']:
    results = []

    for alpha in [0.1, 0.2, 0.5, 1, 2, 5, 10]:
        params = np.load(f'data/{obj}-{setting}/o{alpha}/params.npz')
        R = params['R']

        for trial in range(10):
            yinit = integer_initial_vector(n, R)
            y = integer_initial_vector(n, R)
            pred = integer_initial_vector(n, R)

            for t in range(100):
                with open(f'data/{obj}-{setting}/o{alpha}/{trial*100+t}.pkl', 'rb') as f:
                    G = pickle.load(f)

                # Cold
                start = time.time()
                itr_c, _ = greedy(yinit, G)
                time_c = time.time() - start

                # Learn
                start = time.time()
                itr_p, x = greedy(pred, G)
                time_p = time.time() - start

                # online subgradient descent update
                y += .01 * (R/np.sqrt(n)) * np.sign(x - y)
                y = simplex_projection(y, R)
                pred = (pred*(t+1) + y)/(t+2)

                # Relax
                qsol, time_qc = solve_continuous_quadratic(G, yinit)
                start = time.time()
                itr_q, _ = greedy(qsol, G)
                time_qg = time.time() - start
                time_q = time_qc + time_qg

                results.append([alpha, trial, t, itr_c, itr_p, itr_q, time_c, time_p, time_q, time_qc, time_qg])

                print(obj, alpha, trial, t, itr_c, itr_p, itr_q, time_c, time_p, time_q, time_qc, time_qg)

    # create a DataFrame from the results list
    df = pd.DataFrame(results, columns=['alpha', 'trial', 't', 'itr_c', 'itr_p', 'itr_q', 'time_c', 'time_p', 'time_q', 'time_qc', 'time_qg'])
    #save df as pickle in result folder
    df.to_pickle(f'result/{obj}-{setting}.pkl')

f 0.1 0 0 192 195 7 0.1390531063079834 0.13145875930786133 0.011214971542358398 0.004549980163574219 0.00666499137878418
f 0.1 0 1 192 155 7 0.1306288242340088 0.10421609878540039 0.011595010757446289 0.004879951477050781 0.006715059280395508
f 0.1 0 2 192 114 7 0.13616585731506348 0.08243036270141602 0.011831045150756836 0.004915952682495117 0.006915092468261719
f 0.1 0 3 192 97 7 0.12866616249084473 0.06572294235229492 0.011587858200073242 0.0048520565032958984 0.006735801696777344
f 0.1 0 4 192 100 7 0.12855911254882812 0.06747794151306152 0.011445045471191406 0.00486302375793457 0.006582021713256836
f 0.1 0 5 192 85 7 0.12997984886169434 0.05926179885864258 0.011893987655639648 0.005101919174194336 0.0067920684814453125
f 0.1 0 6 192 83 7 0.1284468173980713 0.056758880615234375 0.029551029205322266 0.004899024963378906 0.02465200424194336
f 0.1 0 7 192 76 7 0.13267993927001953 0.05269002914428711 0.012767314910888672 0.005274057388305664 0.007493257522583008
f 0.1 0 8 192 71 7 0.13